Adapt existing randomization inference code to support power/sample size calculations

In [9]:
# import packages
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.stats as stats
import scipy

from matplotlib import style
from tqdm.auto import tqdm
from importlib import reload

In [10]:
# additional imports

In [11]:
# set pandas preferences
pd.options.display.max_columns=500
pd.options.display.max_colwidth=500
pd.options.display.max_rows=500

In [12]:
# set plot style
style.use("fivethirtyeight")

In [13]:
# initialize tqdm for pandas
tqdm.pandas()

In [14]:
sys.path.append("../../ab_testing_utils/") 

In [15]:
import skewed_metric_utils

In [16]:
import minimum_detectable_effect_size

In [17]:
df = pd.read_csv('../../../data/test_data_for_ri.csv')

In [18]:
df

,name,d,y,y0,y1,id_unit
0,Andy,1,10,NaN,10.0,1
1,Ben,1,5,NaN,5.0,2
2,Chad,1,16,NaN,16.0,3
3,Daniel,1,3,NaN,3.0,4
4,Edith,0,5,5.0,NaN,5
5,Frank,0,7,7.0,NaN,6
6,George,0,8,8.0,NaN,7
7,Hank,0,10,10.0,NaN,8


We assume that the dataset we have is historical:

1. Use randomization inference to generate a null distribution

2. Shift the null to get the right power for the "alternate"

3. The shift amount is the minimum detectable effect size

4. Repeat using historical samples of different sizes

5. Map sample size to expected run time given baseline number of daily observations (roll up to weeks)

6. Map run time --> minimum detectable effect size

In [19]:
ri_ = skewed_metric_utils.RandomizationInference()